In [1]:
import os
import sys
import pandas as pd
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

In [2]:
# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

In [5]:
%%time
# optimize to pull one query by wire centers

# multigig enabled polygons:
# PHNXAZNE - polygon 1179499 - multigig enabled 9/30/22
# PHNXAZSO - polygons 1179547, 1179498, 1179121 - multigig enabled 9/30/22
# ARVDCOMA - polygon 1178832 
# MPLSMNBE - polygon 1178694
# MPLSMNBE - polygon 1179569 - enabled 10/14/22
# PYLPWA01 - polygon 1178144
# GLDNCOMA - polygons 1179687, 1179674 - enabled 10/10/22
# PTLDOR11 - polygon 1180012 - enabled 10/15/22
# DNVRCOMA - polygon 1179864 - enabled 10/15/22
# MPLSMNTF - polygon 1179688 - enabled 10/14/22
# STPLMNBE - polygon 1179553 - enabled 10/14/22
# MPLSMNGE - polygon 1179808 - enabled 10/21/22
# MPLSMNBE - polygon 1179862 - enabled 10/21/22
# MPLSMNBE - polygon 1179836 - enabled 10/21/22
# MPLSMNBE - polygon 1179604 - enabled 10/21/22
# MPLSMNBE - polygon 1179838 - enabled 10/21/22
# MPLSMNBE - polygon 1179887 - enabled 10/21/22
# MPLSMNBE - polygon 1180054 - enabled 10/21/22
# MPLSMNBE - polygon 1180222 - enabled 10/21/22
# MPLSMNTF - polygon 1179157 - enabled 10/21/22
# MPLSMNTF - polygon 1179574 - enabled 10/21/22
# MPLSMNBE - polygon 1180028 - enabled 10/27/22
# GLDNCOMA - polygon 1180254 - enabled 10/27/22
# DNVRCODC - polygon 1179583 - enabled 10/27/22
# MPLSMNGE - polygon 1179976 - enabled 10/28/22




# each polygon will be associated with only one wire center, so only polygon id is needed in the filter

multigig_enabled_polygons_query = """

 WITH LQO AS (
            SELECT  DISTINCT LUID
                    ,WIRE_CENTER_ID
                    ,ADDRESS
                    ,ZIP_CODE
                    ,UNIT
                    ,UNI_POLYGONID        
            
            FROM    LQSMO.LU_QUAL_O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND (UNI_POLYGONID in ('1179499',  -- PHNXAZNE
                                          '1179547', '1179498', '1179121', -- PHNXAZSO
                                          '1178832', -- ARVDCOMA
                                          '1180028', '1178694', '1179569', '1179862', '1179836', '1179604', '1179838', '1179887', '1180054', '1180222', -- MPLSMNBE
                                          '1178144', --PYLPWA01
                                          '1179687', '1179674','1180254', --GLDNCOMA
                                          '1180012', -- PTLDOR11
                                          '1179864', -- DNVRCOMA 
                                          '1179553', -- STPLMNBE 
                                          '1179688','1179157', '1179574',  -- MPLSMNTF
                                          '1179808','1179976', -- MPLSMNGE
                                          '1179583' -- DNVRCODC
                                          )
                        OR WIRE_CENTER_ID = 'MPLSMNBE') -- per request, want all polygons in MPLSMNBE
        
        ),
        LQ AS (
        
        SELECT  Q.LUID
                ,Q.WIRE_CENTER_ID
                ,Q.ADDRESS
                ,Q.HNO
                ,Q.ST
                ,Q.UNIT
                ,Q.FLR
                ,Q.BLDG
                ,Q.COM
                ,Q.ZIP_CODE
                ,Q.TAR_CODE
                ,Q.BORN_ON_DATE
                ,Q.CAF_CENSUS_BLOCK
                ,Q.SERVING_TERMINAL
                ,Q.LU_XBOX
                ,Q.ASSOCIATED_TERMINAL_ID
                ,Q.REMOTE_TERMINAL_ID
                ,Q.REMOTE_TYPE
                ,Q.RES_BUS
                ,Q.LU_STAT

            FROM
                LU_QUAL Q
                JOIN LQO O
                    ON Q.WIRE_CENTER_ID = O.WIRE_CENTER_ID
        ),
        
        LC AS (
            
            SELECT * 
            FROM LU_CIRCUITS
        )
        
        SELECT DISTINCT
                O.LUID AS LU_QUAL_O_LUID
                ,O.UNI_POLYGONID
                ,A.*
                ,COALESCE(B.TELEPHONE_NUMBER,LCO.SUBSCRIBER) as TELEPHONE_OR_SUBSCRIBER
                ,B.LOOP_NAME
                ,B.CIRCUIT_IDENTIFIER
                ,B.SERVICE_CODE
                ,B.SERVICE_TYPE
                ,B.CS_USOC
                ,COALESCE(B.AL_USOC,LCO.HSI_SERVICE_PROFILE) as AL_USOC_OR_HSI_SERVICEPROFILE
                ,B.USER_ID
                ,B.TIME_STAMP
        FROM  LQO O
                JOIN LQ A
                    ON (TRIM(A.ADDRESS)||NVL(A.UNIT||A.FLR||A.BLDG,'.')=O.ADDRESS||NVL(REGEXP_REPLACE(O.UNIT, '^UNIT\s|^UNT\s|^APT\s|^FLR\s|BLDG\s', ''),'.'))
                    AND O.WIRE_CENTER_ID = A.WIRE_CENTER_ID
                LEFT JOIN LC B
                    ON  A.WIRE_CENTER_ID=B.WIRE_CENTER_ID
                    AND A.LUID=B.LUID
                LEFT JOIN LU_CIRCUITS_O LCO
                    ON O.LUID = LCO.LUID
                    AND O.WIRE_CENTER_ID = LCO.WIRE_CENTER_ID
                    AND LCO.SERVICE_FACS_STATUS = 'WKG'   --working customer base

"""

multigig_enabled_polygons_df = pd.read_sql(multigig_enabled_polygons_query, con=conn)

CPU times: total: 172 ms
Wall time: 54.3 s


In [6]:
# check unique wire centers
multigig_enabled_polygons_df['WIRE_CENTER_ID'].value_counts()

MPLSMNBE    3752
MPLSMNGE     989
MPLSMNTF     925
PHNXAZSO     879
GLDNCOMA     484
DNVRCOMA     443
PYLPWA01     342
PHNXAZNE     218
STPLMNBE     213
ARVDCOMA     207
DNVRCODC     175
PTLDOR11      63
Name: WIRE_CENTER_ID, dtype: int64

In [7]:
# separate data by wire center - manually add in wire centers

MPLSMNBE = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'MPLSMNBE']
PHNXAZSO = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'PHNXAZSO']
PYLPWA01 = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'PYLPWA01']
PHNXAZNE = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'PHNXAZNE']
ARVDCOMA = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'ARVDCOMA']
GLDNCOMA = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'GLDNCOMA']
DNVRCOMA = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'DNVRCOMA']
MPLSMNTF = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'MPLSMNTF']
STPLMNBE = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'STPLMNBE']
PTLDOR11 = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'PTLDOR11']
MPLSMNGE = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'MPLSMNGE']
DNVRCODC = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'DNVRCODC']

In [8]:
import time

# export
TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate + "_LU_QUAL_slowsales.xlsx"

# add in new sheets manually if there are new wire centers turning up

with pd.ExcelWriter(excelfilename) as writer:  
    ARVDCOMA.to_excel(writer, sheet_name='ARVDCOMA', index = False)
    MPLSMNBE.to_excel(writer, sheet_name='MPLSMNBE_ALL_POLYGONS', index = False)
    PYLPWA01.to_excel(writer, sheet_name='PYLPWA01', index = False)
    PHNXAZSO.to_excel(writer, sheet_name='PHNXAZSO', index = False)
    PHNXAZNE.to_excel(writer, sheet_name='PHNXAZNE', index = False)
    GLDNCOMA.to_excel(writer, sheet_name='GLDNCOMA', index = False)
    DNVRCOMA.to_excel(writer, sheet_name='DNVRCOMA', index = False)
    MPLSMNTF.to_excel(writer, sheet_name='MPLSMNTF', index = False)
    STPLMNBE.to_excel(writer, sheet_name='STPLMNBE', index = False)
    PTLDOR11.to_excel(writer, sheet_name='PTLDOR11', index = False)
    MPLSMNGE.to_excel(writer, sheet_name='MPLSMNGE', index = False)
    DNVRCODC.to_excel(writer, sheet_name='DNVRCODC', index = False)